In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib as mpl
import os
import numpy as np
from datetime import datetime, timedelta
import math
import orjson
from collections import Counter
from adjustText import adjust_text
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from concurrent.futures import ThreadPoolExecutor, as_completed

plt.rcParams["figure.figsize"] = [25, 10]

In [ ]:
index_list_names = [
    "loadtest-webrtc-2024-kurento-2p",
    "loadtest-webrtc-2024-kurento-8p",
    "loadtest-webrtc-2024-kurento-3p-10s",
    "loadtest-webrtc-2024-kurento-3p-40s",
    "loadtest-webrtc-2024-pion-2p",
    "loadtest-webrtc-2024-pion-8p",
    "loadtest-webrtc-2024-pion-3p-10s",
    "loadtest-webrtc-2024-pion-3p-40s",
]

index_list_full_names = [
    "Kurento, 2 publishers per session",
    "Kurento, 8 publishers per session",
    "Kurento, 3 publishers and 10 subscribers per session",
    "Kurento, 3 publishers and 40 subscribers per session",
    "Pion, 2 publishers per session",
    "Pion, 8 publishers per session",
    "Pion, 3 publishers and 10 subscribers per session",
    "Pion, 3 publishers and 40 subscribers per session",
]  

In [ ]:
def read_file(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return orjson.loads(f.read())
    else:
        return []

def get_index_data(index_list_name, data_type):
    full_user_data = []

    for entry in os.scandir(f'stats/{index_list_name}'):
        if entry.is_dir():
            for sub_entry in os.scandir(entry.path):
                if sub_entry.is_dir():
                    data_path = os.path.join(sub_entry.path, data_type + '.json')
                    if os.path.exists(data_path):
                        data = read_file(data_path)
                        full_user_data.append({
                            'user': sub_entry.name,
                            'session': entry.name,
                            data_type: data
                        })
    return full_user_data


def is_publisher(index, user):
    return ("8p" in index) or (user <= 3)


In [ ]:
def process_user_data(user_data):
    data = []
    for user in user_data:
        for event in user['events']:
            data.append({
                'user': int(user['user'].replace("User", "")),
                'session': int(user['session'].replace("LoadTestSession", "")),
                'event': event['event'] + (f"-{event['connection']}" if 'connection' in event else ""),
                'connection': event['connection'] if 'connection' in event else None,
                'timestamp': event['timestamp'],
                'publishers': 0,
                'subscribers': 0,
                'streams_in': 0,
                'streams_out': 0
            })
    return data

def process_index_list(index_list_name):
    user_data = get_index_data(index_list_name, "events")
    data = process_user_data(user_data)
    events_df = pd.DataFrame(data)
    events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
    events_df = events_df.sort_values(by='timestamp')
    return index_list_name, events_df

events_dfs = {}

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(process_index_list, index_list_name): index_list_name for index_list_name in index_list_names}
    for future in as_completed(futures):
        index_list_name, events_df = future.result()
        events_dfs[index_list_name] = events_df

In [ ]:
def add_to(array, number, user, session, timestamp):
    array.append({
        "number": number,
        "user": user,
        "session": session,
        "timestamp": timestamp
    })
full_publishers_progression = {}
full_subscribers_progression = {}
full_streams_in_progression = {}
full_streams_out_progression = {}
def process_events_df(index_list_name, events_df):
    sessions = events_df['session'].unique()
    publishers_progression = []
    subscribers_progression = []
    streams_in_progression = []
    streams_out_progression = []

    previous_data = {'publishers': 0, 'subscribers': 0, 'streams_in': 0, 'streams_out': 0}
    for session in sessions:
        previous_data[session] = {'publishers': 0, 'subscribers': 0}

    for i, row in events_df.iterrows():
        session = row['session']
        user = row['user']
        event = row['event']
        connection = row['connection']

        if event == 'streamCreated' and connection == 'local':
            if "0s" in index_list_name and user > 3:
                subscribers = previous_data['subscribers'] + 1
                publishers_in_session = previous_data[session]['publishers']
                subscribers_in_session = previous_data[session]['subscribers'] + 1
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['subscribers'] = subscribers
                previous_data[session]['subscribers'] = subscribers_in_session
            else:
                publishers = previous_data['publishers'] + 1
                streams_in = previous_data['streams_in'] + 2
                publishers_in_session = previous_data[session]['publishers'] + 1
                subscribers_in_session = previous_data[session]['subscribers']
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['publishers'] = publishers
                previous_data['streams_in'] = streams_in
                previous_data[session]['publishers'] = publishers_in_session

            previous_data['streams_out'] = streams_out

            add_to(publishers_progression, previous_data['publishers'], user, session, row['timestamp'])
            add_to(subscribers_progression, previous_data['subscribers'], user, session, row['timestamp'])
            add_to(streams_in_progression, previous_data['streams_in'], user, session, row['timestamp'])
            add_to(streams_out_progression, previous_data['streams_out'], user, session, row['timestamp'])

        elif event == 'streamDestroyed' and connection == 'local':
            if "0s" in index_list_name and user > 3:
                subscribers = previous_data['subscribers'] - 1
                publishers_in_session = previous_data[session]['publishers']
                subscribers_in_session = previous_data[session]['subscribers'] - 1
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['subscribers'] = subscribers
                previous_data[session]['subscribers'] = subscribers_in_session
            else:
                publishers = previous_data['publishers'] - 1
                streams_in = previous_data['streams_in'] - 2
                publishers_in_session = previous_data[session]['publishers'] - 1
                subscribers_in_session = previous_data[session]['subscribers']
                streams_out = 2 * publishers_in_session * (publishers_in_session - 1) + 2 * subscribers_in_session * publishers_in_session

                previous_data['publishers'] = publishers
                previous_data['streams_in'] = streams_in
                previous_data[session]['publishers'] = publishers_in_session

            previous_data['streams_out'] = streams_out

            add_to(publishers_progression, previous_data['publishers'], user, session, row['timestamp'])
            add_to(subscribers_progression, previous_data['subscribers'], user, session, row['timestamp'])
            add_to(streams_in_progression, previous_data['streams_in'], user, session, row['timestamp'])
            add_to(streams_out_progression, previous_data['streams_out'], user, session, row['timestamp'])
        
        else:
            events_df.at[i, 'publishers'] = previous_data['publishers']
            events_df.at[i, 'subscribers'] = previous_data['subscribers']
            events_df.at[i, 'streams_in'] = previous_data['streams_in']
            events_df.at[i, 'streams_out'] = previous_data['streams_out']

    publishers_progression = pd.DataFrame(publishers_progression)
    subscribers_progression = pd.DataFrame(subscribers_progression)
    streams_in_progression = pd.DataFrame(streams_in_progression)
    streams_out_progression = pd.DataFrame(streams_out_progression)

    return (index_list_name, publishers_progression, subscribers_progression, streams_in_progression, streams_out_progression)

# Use parallel processing to handle events dataframes
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = [executor.submit(process_events_df, index_list_name, events_dfs[index_list_name]) for index_list_name in index_list_names]
    for future in as_completed(futures):
        (index_list_name, publishers_progression, subscribers_progression, streams_in_progression, streams_out_progression) = future.result()
        full_publishers_progression[index_list_name] = publishers_progression
        full_subscribers_progression[index_list_name] = subscribers_progression
        full_streams_in_progression[index_list_name] = streams_in_progression
        full_streams_out_progression[index_list_name] = streams_out_progression

In [ ]:
# read log file and get specific lines with regex
import re
def process_log_file(log_file):
    regex = re.compile(r'(.*) ERROR .* Participant (.*) in session (.*) failed (\d+) times')
    date_format = "%Y-%m-%d %H:%M:%S.%f %z"
    failed_attempts = []
    
    with open(log_file, 'r') as f:
        for line in f:
            if 'Participant' in line:
                match = regex.search(line)
                if match:
                    failed_attempts.append({
                        'user': match.group(2).replace('User', ''),
                        'session': match.group(3).replace('LoadTestSession', ''),
                        'attempts': int(match.group(4)),  # Ensure attempts is an integer
                        'timestamp': datetime.strptime(match.group(1) + " +0200", date_format),
                        'full_user': f"{match.group(2).replace('User', '')}-{match.group(3).replace('LoadTestSession', '')}",
                    })
    return log_file, pd.DataFrame(failed_attempts)

failures = {}

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(process_log_file, f'logs/{index_list_name}.log'): index_list_name for index_list_name in index_list_names}
    for future in as_completed(futures):
        log_file, df = future.result()
        index_list_name = log_file.split('/')[-1].replace('.log', '')
        failures[index_list_name] = df

In [ ]:
# for test, events_df in events_dfs.items():
#     events_df = events_df.sort_values(by=["session", "user", "timestamp"])
#     fig, ax = plt.subplots()
#     event_types = events_df['event'].unique()
#     event_types.sort()
#     colors = mpl.colormaps.get_cmap('tab20')(np.linspace(0, 1, len(event_types)))
#     event_color = dict(zip(event_types, colors))
#     y_labels = []
#     y_ticks = 0
#     for session in events_dfs[test]['session'].unique():
#         events_df_session = events_df[events_df['session'] == session]
#         for user in events_df_session['user'].unique():
#             events_df_user = events_df_session[events_df_session['user'] == user]
#             y_ticks += 1
#             y_pos = [y_ticks] * len(events_df_user)
#             ax.scatter(events_df_user['timestamp'], y_pos, c=[event_color[event] for event in events_df_user['event']])
#             y_labels.append(f'{user} - {session}')

#     # add legend
#     legend_labels = []
#     for event, color in event_color.items():
#         legend_labels.append(plt.Line2D([0], [0], marker='o', color='w', label=event, markerfacecolor=color))
#     ax.legend(handles=legend_labels, loc='upper left')
#     fig.suptitle(f'{test}')
#     # y labels should be user - session
#     ax.set_yticks(range(1, y_ticks + 1))
#     ax.set_yticklabels(y_labels)


In [ ]:
# for test in events_dfs.keys():
#     events_df = events_dfs[test]
#     events_df = events_df.sort_values(by=["session", "user", "timestamp"])
#     # events_df = events_df[
#     #     (events_df['event'] == 'connectionStart') |
#     #     # (events_df['event'] == 'signalConnected') |
#     #     # (events_df['event'] == 'connectedPublisher') |
#     #     ((events_df['event'] == 'streamCreated') & (events_df['connection'] == 'local')) |
#     #     ((events_df['event'] == 'streamDestroyed') & (events_df['connection'] == 'local')) |
#     #     (events_df['event'] == 'connectionEnd')
#     # ]
#     events_df = events_df[
#         ~(events_df['event'] == 'publisherStartSpeaking-local') & ~(events_df['event'] == 'publisherStopSpeaking-local') &
#         ~(events_df['event'] == 'accessAllowed-local')
#     ]
#     fig, ax = plt.subplots()
#     event_types = events_df['event'].unique()
#     event_types.sort()
#     colors = mpl.colormaps.get_cmap('tab20')(np.linspace(0, 1, len(event_types)))
#     event_color = dict(zip(event_types, colors))
#     y_labels = []
#     y_ticks = 0
#     for session in events_df['session'].unique():
#         events_df_session = events_df[events_df['session'] == session]
#         for user in events_df_session['user'].unique():
#             events_df_user = events_df_session[events_df_session['user'] == user]
#             y_ticks += 1
#             y_pos = [y_ticks] * len(events_df_user)
#             ax.scatter(events_df_user['timestamp'], y_pos, c=[event_color[event] for event in events_df_user['event']])
#             y_labels.append(f'{user} - {session}')


#     # add legend
#     legend_labels = []
#     for event, color in event_color.items():
#         legend_labels.append(plt.Line2D([0], [0], marker='o', color='w', label=event, markerfacecolor=color))
#     ax.legend(handles=legend_labels, loc='upper left')
#     fig.suptitle(f'{test}')
#     # y labels should be user - session
#     ax.set_yticks(range(1, y_ticks + 1))
#     ax.set_yticklabels(y_labels)

In [ ]:
df_cpu_list = []
for index in index_list_names:
    df_cpu = pd.read_csv(f"dfs_final/{index}-medianode.csv")
    df_cpu = df_cpu.drop(columns=["memory"]).dropna()
    df_cpu["@timestamp"] = pd.to_datetime(df_cpu["@timestamp"], format="ISO8601")
    df_cpu_list.append(df_cpu)

In [ ]:
index_qoe_df_list = []
for index in index_list_names:
    index_qoe_df = pd.DataFrame()
    for entry in os.scandir(f"qoe/{index}"):
        if entry.is_file():
            with open(f"qoe/{index}/{entry.name}", 'r') as f:
                qoe = orjson.loads(f.read())
            qoe_df = pd.DataFrame.from_dict(qoe)
            info = entry.name.split("_")[0].split("-")
            session = info[1]
            userRecording = info[2]
            userBeingRecorded = info[3]
            qoe_df["userRecording"] = userRecording
            qoe_df["userBeingRecorded"] = userBeingRecorded
            qoe_df["session"] = session
            if index_qoe_df.empty:
                index_qoe_df = qoe_df
            else:
                index_qoe_df = pd.concat([index_qoe_df, qoe_df])
    index_qoe_df_list.append(index_qoe_df)


seconds_per_fragment = 17
seconds_per_padding = 2
seconds_per_video = seconds_per_fragment - seconds_per_padding
def set_qoe_times(df, user_1_start_connection_time, user_2_start_connection_time):
    cut_index_0_time = 5 # recording stats 5 seconds after connection is successful
    max_time = max(user_1_start_connection_time, user_2_start_connection_time)
    time_fn = lambda cut_index: max_time + timedelta(seconds=(cut_index * seconds_per_fragment) + cut_index_0_time).total_seconds()
    df.loc[:, "timestamp"] = df["cut_index"].apply(time_fn)


In [ ]:
with open(f"dfs_final/batches.json", 'r') as f:
    batches = orjson.loads(f.read())

In [ ]:
user_events_obj = {}

def get_connection_times(events):
    connection_times_recorded = []
    recorded_media_events = events[
        (events["event"] != "connectionStart") &
        (events["event"] != "connectionFail")
    ]
    for m, row in recorded_media_events.iterrows():
        timestamp = row["timestamp"]
        con_time = (timestamp - start_time).total_seconds()
        connection_times_recorded.append(con_time)
    return connection_times_recorded


def get_failure_times(events):
    connection_times_recorded = []
    recorded_media_events = events[
        events["event"] == "connectionFail"
    ]
    for m, row in recorded_media_events.iterrows():
        timestamp = row["timestamp"]
        con_time = (timestamp - start_time).total_seconds()
        connection_times_recorded.append(con_time)
    return connection_times_recorded


connection_stats_global = pd.DataFrame()
df_cpu_filtered_list = []
start_end_times = {}
for i, index_list_name in enumerate(index_list_names):
    #stats_df = get_index_data(index_list_name, "stats")
    events_df = events_dfs[index_list_name]
    events_df = events_df.sort_values(by=["session", "user", "timestamp"])
    start_time = events_df["timestamp"].min()
    end_time = events_df["timestamp"].max()
    test_duration = (end_time - start_time).total_seconds()
    start_end_times[index_list_name] = (start_time, end_time)
    cpu = df_cpu_list[i]
    cpu_filtered = cpu[(cpu["@timestamp"] >= start_time) & (cpu["@timestamp"] <= end_time)]
    cpu_filtered.loc[:, "@timestamp"] = cpu_filtered["@timestamp"].apply(lambda x: (x - start_time).total_seconds())
    cpu_filtered = cpu_filtered.sort_values(by="@timestamp")
    df_cpu_filtered_list.append(cpu_filtered)
    index_qoe_df = index_qoe_df_list[i]
    sessions = events_df["session"].unique()
    user_events_obj[index_list_name] = {}
    for session in sessions:
        users = events_df[events_df["session"] == session]["user"].unique()
        for user in users:
            full_user = f"{user}-{session}"
            user_events = events_df[
                (events_df["session"] == session) & (events_df["user"] == user)
            ]
            connection_starts = user_events[user_events["event"] == "connectionStart"]
            failures_user = failures[index_list_name][
                (failures[index_list_name]["full_user"] == full_user) &
                (failures[index_list_name]["timestamp"] >= start_time) &
                (failures[index_list_name]["timestamp"] <= end_time)
            ].copy()
            failures_user["event"] = "connectionFail"
            failures_user["recoverable"] = False
            if "kurento" in index_list_name:
                if is_publisher(index_list_name, user):
                    stream_created_locals = user_events[
                        user_events["event"] == "streamCreated-local"
                    ]
                    stream_playing_locals = user_events[
                        user_events["event"] == "streamPlaying-local"
                    ]
                else:
                    # we choose the first stream created as confirmation of media connection if the user is subscriber
                    stream_created_locals = user_events[
                        user_events["event"].str.startswith("streamCreated")
                    ]
                    stream_playing_locals = user_events[
                        user_events["event"].str.startswith("streamPlaying")
                    ]
                # combine connection starts and stream created locals and failures_user
                combined = pd.concat(
                    [
                        connection_starts,
                        stream_created_locals,
                        stream_playing_locals,
                        failures_user,
                    ]
                )
                # if there is more than one stream created and a stream playing, we only keep the first one (marks connection success)
                combined = combined.sort_values(by="timestamp")
                prev_row_event = ""
                for r, row in combined.iterrows():
                    is_prev_row_connected = prev_row_event.startswith("streamCreated") or prev_row_event.startswith("streamPlaying")
                    is_current_row_connected = row["event"].startswith("streamCreated") or row["event"].startswith("streamPlaying")
                    if is_prev_row_connected and is_current_row_connected:
                        combined = combined.drop(r)
                    prev_row_event = row["event"]
            else:
                reconnecting_events = user_events[user_events["event"].str.contains("Reconnecting")].copy()
                if not reconnecting_events.empty:
                    reconnecting_events["event"] = "connectionFail"
                    reconnecting_events["recoverable"] = True
                    failures_user = pd.concat([failures_user, reconnecting_events])
                # combine connection starts and stream created locals and failures_user
                if is_publisher(index_list_name, user):
                    stream_created_locals = user_events[
                        user_events["event"] == "LocalTrackPublished-local"
                    ]
                    stream_created_locals = stream_created_locals.sort_values(by="timestamp")
                    # The second one means the publisher has fully connected
                    stream_created_locals = stream_created_locals.iloc[::2, :]
                else:
                    # we choose the first stream created as confirmation of media connection if the user is subscriber
                    stream_created_locals = user_events[
                        user_events["event"].str.startswith("TrackSubscribed")
                    ]
                # combine connection starts and stream created locals and failures_user
                combined = pd.concat(
                    [connection_starts, stream_created_locals, failures_user]
                )
                combined = combined.sort_values(by="timestamp")
                for r, row in combined.iterrows():
                    is_prev_row_connected = prev_row_event.startswith("TrackSubscribed")
                    is_current_row_connected = row["event"].startswith("TrackSubscribed")
                    if is_prev_row_connected and is_current_row_connected:
                        combined = combined.drop(r)
                    prev_row_event = row["event"]
            index_qoe_df_filtered = index_qoe_df[
                (index_qoe_df["session"] == "LoadTestSession" + str(session)) &
                (index_qoe_df["userRecording"] == "User" + str(user))
            ].copy()
            index_qoe_df_filtered = index_qoe_df_filtered.sort_values(by="cut_index")
            user_events_obj[index_list_name][full_user] = {}
            user_events_obj[index_list_name][full_user]["combined"] = combined
            user_events_obj[index_list_name][full_user]["qoe"] = index_qoe_df_filtered
            user_events_obj[index_list_name][full_user]["user"] = user
            user_events_obj[index_list_name][full_user]["session"] = session

In [ ]:
def get_connection_time_intervals(user_events, start_time, test_duration):
    connection_time_intervals = {}
    users = user_events.keys()
    for username in users:
        group = user_events[username]['combined'].copy()
        group["timestamp"] = group["timestamp"].apply(lambda x: (x - start_time).total_seconds())
        connection_success_times = group.loc[~((group['event'] == 'connectionFail') | (group['event'] == 'connectionStart'))]['timestamp']
        connection_fail_times = group.loc[group['event'] == 'connectionFail']['timestamp']

        intervals = []
        for success_time in connection_success_times:
            fail_time = connection_fail_times[connection_fail_times > success_time].min()
            if pd.isnull(fail_time):
                intervals.append((success_time, test_duration))
            else:
                intervals.append((success_time, min(fail_time, test_duration)))

        connection_time_intervals[username] = intervals

    return connection_time_intervals

def intersect_intervals(intervals1, intervals2):
    intersection = []
    for interval1 in intervals1:
        for interval2 in intervals2:
            start = max(interval1[0], interval2[0])
            end = min(interval1[1], interval2[1])
            if start < end:
                intersection.append((start, end))
    return intersection

intersections = {}
connection_time_intervals = {}
for index_list_name in index_list_names:
    intersections[index_list_name] = {}
    start_time, end_time = start_end_times[index_list_name]
    test_duration = (end_time - start_time).total_seconds()
    connection_time_intervals_index = get_connection_time_intervals(user_events_obj[index_list_name], start_time, test_duration)
    connection_time_intervals[index_list_name] = connection_time_intervals_index
    usernames = list(connection_time_intervals_index.keys())
    for i in range(len(usernames)):
        for j in range(i + 1, len(usernames)):
            _, session1 = usernames[i].split("-")
            _, session2 = usernames[j].split("-")
            if session1 == session2:
                user1_intervals = connection_time_intervals_index[usernames[i]]
                user2_intervals = connection_time_intervals_index[usernames[j]]
                intersection = intersect_intervals(user1_intervals, user2_intervals)
                if not usernames[i] in intersections[index_list_name]:
                    intersections[index_list_name][usernames[i]] = {}
                intersections[index_list_name][usernames[i]][usernames[j]] = intersection
                if not usernames[j] in intersections[index_list_name]:
                    intersections[index_list_name][usernames[j]] = {}
                intersections[index_list_name][usernames[j]][usernames[i]] = intersection



In [ ]:
plt.rcParams["figure.figsize"] = [40, 35]

colors_list = ["black", "orange", "yellow", "green", "darkblue"]
bounds = [0.2, 0.4, 0.6, 0.8, 1.0]
norm_bounds = [(b - min(bounds)) / (max(bounds) - min(bounds) ) for b in bounds]
success_cmap = mcolors.LinearSegmentedColormap.from_list("success", list(zip(norm_bounds, colors_list)))
normalize = mcolors.Normalize(vmin=min(bounds), vmax=max(bounds))

def get_con_type(next_color):
    if next_color == "magenta":
        return "connectionStart"
    elif next_color == "green":
        return "connectionSuccess"
    elif next_color == "red":
        return "connectionFail"

def calc_stats(cpu_filtered, prev_time, current_time, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats):
    total_duration = current_time - prev_time
    cpu_subset = cpu_filtered[(cpu_filtered['@timestamp'] >= prev_time) & (cpu_filtered['@timestamp'] <= current_time)]
    con_type = get_con_type(next_color)
    stats = {
        "type": con_type,
        "start_time": prev_time,
        "end_time": current_time,
        "total_duration": total_duration,
        "attempt": prev_attempt,
        "batch": n_batch,
        "user": user,
        "session": session,
        "full_user": full_user
    }
    # if there is no cpu data, add the next cpu data available
    if cpu_subset.empty:
        # seek closest cpu data to start_time or end_time
        cpu_prev_df = cpu_filtered[cpu_filtered['@timestamp'] < prev_time]
        cpu_next_df = cpu_filtered[cpu_filtered['@timestamp'] > current_time]
        # if one of the dfs is empty, we take the other
        if cpu_prev_df.empty:
            cpu_next = cpu_next_df.iloc[0]
            cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_next['@timestamp']]
        elif cpu_next_df.empty:
            cpu_prev = cpu_prev_df.iloc[-1]
            cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_prev['@timestamp']]
        else:
            cpu_next = cpu_next_df.iloc[0]
            cpu_prev = cpu_prev_df.iloc[-1]
            cpu_prev_diff = prev_time - cpu_prev['@timestamp']
            cpu_next_diff = cpu_next['@timestamp'] - current_time
            if cpu_prev_diff < cpu_next_diff:
                cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_prev['@timestamp']]
            else:
                cpu_subset = cpu_filtered[cpu_filtered['@timestamp'] == cpu_next['@timestamp']]
    stats["cpu_mean"] = cpu_subset['cpu'].mean()
    stats["cpu_start"] = cpu_subset['cpu'].iloc[0]
    stats["cpu_end"] = cpu_subset['cpu'].iloc[-1]
    stats["cpu_median"] = cpu_subset['cpu'].median()
    stats["cpu_max"] = cpu_subset['cpu'].max()
    stats["cpu_min"] = cpu_subset['cpu'].min()
    connection_stats.append(stats)
    return connection_stats

def paint_bar(prev_time, current_time, y_ticks, prev_state, next_color):
    total_duration = current_time - prev_time
    yrange = (y_ticks - 0.5, 0.8)
    if prev_state != "connectionSuccess":
        segment = [(prev_time, total_duration)]
        ax.broken_barh(segment, yrange, facecolors=next_color)

def paint_success_bar(prev_time, current_time, y_ticks):
    total_duration = current_time - prev_time
    yrange = (y_ticks - 0.5, 0.8)
    segment = [(prev_time, total_duration)]
    ax.broken_barh(segment, yrange, facecolors=success_cmap(norm_bounds[0]))

for i, index_list_name in enumerate(index_list_names):
    fig, ax = plt.subplots()
    y_labels = []
    y_ticks = 0
    user_count = 0
    connection_stats = []
    batch_info = batches[index_list_name]
    cpu_filtered = df_cpu_filtered_list[i]
    start_time, end_time = start_end_times[index_list_name]
    test_duration = (end_time - start_time).total_seconds()
    for full_user, full_user_obj in user_events_obj[index_list_name].items():
        user_count += 1
        if user_count <= batch_info["startingParticipants"]:
            n_batch = 0
        else:
            n_batch = math.floor((user_count - batch_info["startingParticipants"] - 1) / batch_info["batchSize"]) + 1
        user_split = full_user.split("-")
        user = user_split[0]
        session = user_split[1]
        y_labels.append(full_user)
        y_ticks += 1

        next_color = "white"
        prev_time = None
        prev_state = None
        prev_attempt = 0
        combined = full_user_obj["combined"]
        qoe = full_user_obj["qoe"]
        con_segments = connection_time_intervals[index_list_name][full_user]
        user_intersections = intersections[index_list_name][full_user]
        # connection success
        for segment in con_segments:
            paint_success_bar(segment[0], segment[1], y_ticks)
        # other connection events
        for r, row in combined.iterrows():
            current_time = (row["timestamp"] - start_time).total_seconds()
            if prev_time is not None:
                paint_bar(prev_time, current_time, y_ticks, prev_state, next_color)
                calc_stats(cpu_filtered, prev_time, current_time, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats)
            connectionSuccessful = (
                row["event"].startswith("streamCreated")
                or row["event"].startswith("streamPlaying")
                or row["event"].startswith("LocalTrackPublished")
                or row["event"].startswith("TrackSubscribed")
            )
            if row["event"] == "connectionStart":
                next_color = "magenta"
                prev_state = "connectionStart"
            elif connectionSuccessful:
                next_color = "green"
                prev_state = "connectionSuccess"
            else:
                next_color = "red"
                prev_attempt = row["attempts"]
                prev_state = "connectionFail"
            prev_time = current_time
        if prev_time is not None:
            paint_bar(prev_time, test_duration, y_ticks, prev_state, next_color)
            calc_stats(cpu_filtered, prev_time, test_duration, prev_attempt, n_batch, user, session, full_user, next_color, connection_stats)
    connection_stats_df = pd.DataFrame(connection_stats)
    # add all rows to connection_stats_global
    if connection_stats_global.empty:
        connection_stats_global = connection_stats_df
        connection_stats_global["index"] = index_list_name
    else:
        connection_stats_df["index"] = index_list_name
        connection_stats_global = pd.concat([connection_stats_global, connection_stats_df])

    ax.set_yticks(range(1, y_ticks + 1))
    ax.set_yticklabels(y_labels)
    ax.set_title(f"Connection Progression - {index_list_full_names[i]}", pad=10)
    # y axis label
    ax.set_ylabel("User - Session")
    if user_count >= 150:
        # remove y labels
        ax.set_yticklabels([])
    # x axis label
    ax.set_xlabel("Time (s)")
    ax.set_xlim(-5, test_duration + 5)
    # grid
    ax.grid(True)
    # x axis ticks every 10 seconds
    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
    # cpu in new axis
    ax2 = ax.twinx()
    ax2.set_ylabel("CPU (%)")
    # ax2 y axis should go from 0 to 1
    ax2.set_ylim(0, 1.01)
    ax2.set_yticks(np.arange(0, 1.01, 0.05))
    ax2.plot(cpu_filtered["@timestamp"], cpu_filtered["cpu"], color="black", marker="o", linewidth=4, markersize=10)
    # create legend
    legend_labels = [
        plt.Line2D([0], [0], color="magenta", lw=4, label="Connection Start"),
        plt.Line2D([0], [0], color="indigo", lw=4, label="Stream Created/Playing, No Video"),
        plt.Line2D([0], [0], color="darkblue", lw=4, label="Excellent QoE"),
        plt.Line2D([0], [0], color="green", lw=4, label="Good QoE"),
        plt.Line2D([0], [0], color="yellow", lw=4, label="Fair QoE"),
        plt.Line2D([0], [0], color="orange", lw=4, label="Poor QoE"),
        plt.Line2D([0], [0], color="black", lw=4, label="Bad QoE"),
        plt.Line2D([0], [0], color="red", lw=4, label="Failed Connection"),
        plt.Line2D([0], [0], color="black", lw=4, label="CPU", marker="o", markersize=10)
    ]

    ax.legend(handles=legend_labels, loc="upper left")

In [ ]:
cpu_start_durations = []
def calculate_connection_start_data(connection_stats):
    connection_stats = connection_stats[connection_stats["type"] == "connectionStart"].drop(columns=["type", "index", "user", "session"])
    # connection_stats = connection_stats.sort_values(by="start_time").reset_index().drop(columns=["index"])
    connection_stats = connection_stats.sort_values(by="cpu_median").reset_index().drop(columns=["index"])
    connection_stats = connection_stats[["full_user", "attempt", "batch", "total_duration", "cpu_median", "cpu_mean", "cpu_max", "cpu_min"]]

    # average, median, min, max total duration when cpu_median is below 0.8
    connection_stats_filtered = connection_stats[connection_stats["cpu_median"] < 0.8]
    mean80 = connection_stats_filtered["total_duration"].mean()
    median80 = connection_stats_filtered["total_duration"].median()
    min80 = connection_stats_filtered["total_duration"].min()
    max80 = connection_stats_filtered["total_duration"].max()
    cpu_start_durations.append({
        "test case": index,
        "threshold": "Median CPU < 0.8",
        "median connection start duration (seconds)": median80,
        "mean connection start duration (seconds)": mean80,
        "min connection start duration (seconds)": min80,
        "max connection start duration (seconds)": max80
    })
    # average, median, min, max total duration when cpu_median is between 0.8 and 0.9
    connection_stats_filtered = connection_stats[(connection_stats["cpu_median"] >= 0.8) & (connection_stats["cpu_median"] < 0.9)]
    mean89 = connection_stats_filtered["total_duration"].mean()
    median89 = connection_stats_filtered["total_duration"].median()
    min89 = connection_stats_filtered["total_duration"].min()
    max89 = connection_stats_filtered["total_duration"].max()
    cpu_start_durations.append({
        "test case": index,
        "threshold": "0.8 <= Median CPU < 0.9",
        "median connection start duration (seconds)": median89,
        "mean connection start duration (seconds)": mean89,
        "min connection start duration (seconds)": min89,
        "max connection start duration (seconds)": max89
    })
    # average, median, min, max total duration when cpu_median is between 0.9 and 1
    connection_stats_filtered = connection_stats[(connection_stats["cpu_median"] >= 0.9) & (connection_stats["cpu_median"] <= 1)]
    mean90 = connection_stats_filtered["total_duration"].mean()
    median90 = connection_stats_filtered["total_duration"].median()
    min90 = connection_stats_filtered["total_duration"].min()
    max90 = connection_stats_filtered["total_duration"].max()
    cpu_start_durations.append({
        "test case": index,
        "threshold": "0.9 <= Median CPU <= 1",
        "median connection start duration (seconds)": median90,
        "mean connection start duration (seconds)": mean90,
        "min connection start duration (seconds)": min90,
        "max connection start duration (seconds)": max90
    })

In [ ]:
# TODO: Test estadísticos (Wilconxon)
# Connection starts
for index in index_list_names:
        connection_stats = connection_stats_global[connection_stats_global["index"] == index]
        calculate_connection_start_data(connection_stats)

cpu_start_durations_df = pd.DataFrame(cpu_start_durations)
display(cpu_start_durations_df)
cpu_start_durations = []

In [ ]:
# Connection start of successful connections
for index in index_list_names:
    connection_stats = connection_stats_global[connection_stats_global["index"] == index]
    # remove all connectionStart that are not followed by a connectionSuccess for that full_user and attempt
    for i, row in connection_stats.iterrows():
        if row["type"] == "connectionStart":
            next_row = connection_stats[
                (connection_stats["full_user"] == row["full_user"])
                & (connection_stats["attempt"] == row["attempt"])
                & (connection_stats["start_time"] > row["start_time"])
            ]
            if next_row.empty or next_row.iloc[0]["type"] != "connectionSuccess":
                connection_stats = connection_stats.drop(i)

    calculate_connection_start_data(connection_stats)

cpu_start_durations_df = pd.DataFrame(cpu_start_durations)
display(cpu_start_durations_df)
cpu_start_durations = []

In [ ]:
cpu_start_durations = []
for index in index_list_names:
    if "10s" in index or "40s" in index:
        connection_stats = connection_stats_global[connection_stats_global["index"] == index]
        # remove all connectionStart that are not followed by a connectionSuccess for that full_user and attempt
        for i, row in connection_stats.iterrows():
            if row["type"] == "connectionStart":
                if int(row["user"]) > 3:
                    connection_stats = connection_stats.drop(i)
                else:
                    next_row = connection_stats[
                        (connection_stats["full_user"] == row["full_user"])
                        & (connection_stats["attempt"] == row["attempt"])
                        & (connection_stats["start_time"] > row["start_time"])
                    ]
                    if next_row.empty or next_row.iloc[0]["type"] != "connectionSuccess":
                        connection_stats = connection_stats.drop(i)
        calculate_connection_start_data(connection_stats)

cpu_start_durations_df = pd.DataFrame(cpu_start_durations)
display(cpu_start_durations_df)
cpu_start_durations = []

In [ ]:
cpu_start_durations = []
for index in index_list_names:
    if "10s" in index or "40s" in index:
        connection_stats = connection_stats_global[connection_stats_global["index"] == index]
        # remove all connectionStart that are not followed by a connectionSuccess for that full_user and attempt
        for i, row in connection_stats.iterrows():
            if row["type"] == "connectionStart":
                if int(row["user"]) <= 3:
                    connection_stats = connection_stats.drop(i)
                else:
                    next_row = connection_stats[
                        (connection_stats["full_user"] == row["full_user"])
                        & (connection_stats["attempt"] == row["attempt"])
                        & (connection_stats["start_time"] > row["start_time"])
                    ]
                    if next_row.empty or next_row.iloc[0]["type"] != "connectionSuccess":
                        connection_stats = connection_stats.drop(i)
        calculate_connection_start_data(connection_stats)

cpu_start_durations_df = pd.DataFrame(cpu_start_durations)
display(cpu_start_durations_df)
cpu_start_durations = []

In [ ]:
# 3 types of stats
# webrtcStats: jitter etc.
# inbound: qoe calculated via inbound metrics
# outbound: qoe calculated via outbound metrics
# at the moment we only care about webrtcStats

stats_dict = {
    "inbound": {},
    "outbound": {},
    "remote_inbound": {},
    "remote_outbound": {}
}

def process_stats(index_list_name):
    stats = get_index_data(index_list_name, "stats")
    for stat in stats:
        if 'stats' in stat:
            in_stats = stat['stats']
            for s in in_stats:
                # localUser = s['user']
                # localSession = s['session']
                if 'webrtcStats' in s:
                    webrtc_stats = s['webrtcStats']
                    for ws in webrtc_stats:
                        if ('event' in ws) and (ws['event'] == 'stats'):
                            yield ws

def add_remote_peer(data, remote_peer):
    for d in data:
        d['peerId'] = remote_peer


def separate_stats(index_list_name):
    def process_data(data, peer_id, direction):
        if data:
            add_remote_peer(data, peer_id)
            if index_list_name not in stats_dict[direction]:
                stats_dict[direction][index_list_name] = []
            stats_dict[direction][index_list_name].extend(data)

    for r in process_stats(index_list_name):
        remotePeer = r['peerId']
        if 'data' in r:
            d = r['data']
            local_audio = d['audio']
            local_video = d['video']
            remote_audio = d['remote']['audio']
            remote_video = d['remote']['video']

            process_data(local_audio['inbound'], remotePeer, 'inbound')
            process_data(local_audio['outbound'], remotePeer, 'outbound')
            process_data(local_video['inbound'], remotePeer, 'inbound')
            process_data(local_video['outbound'], remotePeer, 'outbound')
            process_data(remote_audio['inbound'], remotePeer, 'remote_inbound')
            process_data(remote_audio['outbound'], remotePeer, 'remote_outbound')
            process_data(remote_video['inbound'], remotePeer, 'remote_inbound')
            process_data(remote_video['outbound'], remotePeer, 'remote_outbound')

    for key in ['inbound', 'outbound', 'remote_inbound', 'remote_outbound']:
        stats_dict[key][index_list_name] = pd.DataFrame(stats_dict[key][index_list_name])
        stats_dict[key][index_list_name]['peerId'] = remotePeer

# Using ThreadPoolExecutor for concurrent processing
with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(separate_stats, index_list_name): index_list_name for index_list_name in index_list_names}
    for future in as_completed(futures):
        future.result()


In [ ]:
#remove all entries where audio.inbound is an empty list
def not_empty_list(x):
    return x != []
normalized_inbound = normalized_qos[normalized_qos["audio.inbound"].apply(not_empty_list)]

# keep only audio.inbound, video.inbound, remote.audio.outbound, remote.video.outbound and peerID
normalized_inbound = normalized_inbound[["audio.inbound", "video.inbound", "remote.audio.outbound", "remote.video.outbound", "peerId"]]

def expand_column(df, column_name):
    expanded = df[column_name].apply(pd.Series).stack().reset_index(level=1, drop=True)
    expanded = pd.json_normalize(expanded)
    return expanded

audio_inbound = expand_column(normalized_inbound, "audio.inbound")
video_inbound = expand_column(normalized_inbound, "video.inbound")
remote_audio_outbound = expand_column(normalized_inbound, "remote.audio.outbound")
remote_video_outbound = expand_column(normalized_inbound, "remote.video.outbound")

In [ ]:
plt_audio_inbound = audio_inbound.loc[:, ["timestamp", "packetRate", "bitrate"]]
plt_audio_inbound["timestamp"] = pd.to_datetime(plt_audio_inbound["timestamp"], unit="ms")

cpu = df_cpu_list[0]
cpu = cpu.drop(columns=["memory"]).dropna()
cpu["@timestamp"] = pd.to_datetime(cpu["@timestamp"], format="ISO8601").dt.tz_localize(None)

publishers_progression["timestamp"] = publishers_progression["timestamp"].dt.tz_localize(None)

# here min_ts is calculated as the minimum datetime of the 3 dataframes
# cpu are time aware while plt_audio_inbound is not
min_ts = pd.concat([cpu["@timestamp"], publishers_progression["timestamp"], plt_audio_inbound["timestamp"]]).min()

cpu_td = pd.DataFrame(columns=["timestamp", "cpu"])

cpu_td["timestamp"] = (cpu["@timestamp"] - min_ts).dt.total_seconds()
cpu_td["cpu"] = cpu["cpu"]

user_td = pd.DataFrame(columns=["timestamp", "publishers"])
user_td["timestamp"] = (publishers_progression["timestamp"] - min_ts).dt.total_seconds()
user_td["publishers"] = publishers_progression["number"]
user_td["label"] = publishers_progression["user"] + "-" + publishers_progression["session"]

plt_audio_inbound["timestamp"] = (plt_audio_inbound["timestamp"] - min_ts).dt.total_seconds()

In [ ]:
plt.rcParams["figure.figsize"] = [25, 10]

fig, ax = plt.subplots()
ax.plot(cpu_td["timestamp"], cpu_td["cpu"], label="CPU usage", color="black", marker="o", linestyle="--", alpha=0.2)
ax.set_xlabel("time")
ax.set_ylabel("CPU usage (%)")
ax.grid()
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
#ax.set_xlim(100, 110)
ax2 = ax.twinx()
ax2.set_ylabel("users")
ax2.spines["right"].set_position(("axes", 1.025))
ax2.plot(user_td["timestamp"], user_td["publishers"], label="users", color="red", marker="o", linestyle="-")
ax2.yaxis.set_major_locator(ticker.MultipleLocator(1))

# Annotate users plot with offset to prevent overlap
texts = [ax2.text(x, y, label) for x, y, label in zip(user_td["timestamp"], user_td["publishers"], user_td["label"])]
adjust_text(texts, arrowprops=dict(arrowstyle="->", color='black'))

# Commented out third y-axis for bitrate
# ax3 = ax.twinx()
# ax3.set_ylabel("bitrate")
# ax3.spines["right"].set_position(("axes", 1.075))
# ax3.plot(plt_audio_inbound["timestamp"], plt_audio_inbound["bitrate"], label="bitrate", linestyle="-")

fig.legend(loc='upper right')
fig.suptitle("Users over time")

In [ ]:
plt.rcParams["figure.figsize"] = [25, 10]

fig, ax = plt.subplots()
ax.plot(cpu_td["timestamp"], cpu_td["cpu"], label="CPU usage", color="black", marker="o", linestyle="--")
ax.set_xlabel("time")
ax.set_ylabel("CPU usage (%)")
ax.grid()
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax2 = ax.twinx()
ax2.set_ylabel("users")
ax2.spines["right"].set_position(("axes", 1.025))
ax2.plot(user_td["timestamp"], user_td["publishers"], label="users", color="red", marker="o", linestyle="-")
ax3 = ax.twinx()
ax3.set_ylabel("bitrate")
ax3.spines["right"].set_position(("axes", 1.075))
ax3.plot(plt_audio_inbound["timestamp"], plt_audio_inbound["packetRate"], label="packet rate", linestyle="-")

fig.legend()
fig.suptitle("Inbound audio rate over time")

In [ ]:
#remove all entries where audio.inbound is an empty list
def not_empty_list(x):
    return x != []
normalized_outbound = normalized_qos[normalized_qos["audio.outbound"].apply(not_empty_list)]

# keep only audio.inbound, video.inbound, remote.audio.outbound, remote.video.outbound and peerID
normalized_outbound = normalized_outbound[["audio.outbound", "video.outbound", "remote.audio.inbound", "remote.video.inbound", "peerId"]]

def expand_column(df, column_name):
    expanded = df[column_name].apply(pd.Series).stack().reset_index(level=1, drop=True)
    expanded = pd.json_normalize(expanded)
    return expanded

audio_outbound = expand_column(normalized_outbound, "audio.outbound")
video_outbound = expand_column(normalized_outbound, "video.outbound")
remote_audio_inbound = expand_column(normalized_outbound, "remote.audio.inbound")
remote_video_inbound = expand_column(normalized_outbound, "remote.video.inbound")

In [ ]:
plt_remote_audio_inbound = remote_audio_inbound.loc[:, ["timestamp", "roundTripTime"]]
plt_remote_audio_inbound["timestamp"] = pd.to_datetime(plt_remote_audio_inbound["timestamp"], unit="ms")

min_ts = pd.concat([cpu["@timestamp"], user_join["@timestamp"], plt_remote_audio_inbound["timestamp"]]).min()

cpu_td["timestamp"] = (cpu["@timestamp"] - min_ts).dt.total_seconds()

user_td["timestamp"] = (user_join["@timestamp"] - min_ts).dt.total_seconds()

plt_remote_audio_inbound["timestamp"] = (plt_remote_audio_inbound["timestamp"] - min_ts).dt.total_seconds()

In [ ]:
plt.rcParams["figure.figsize"] = [25, 10]

fig, ax = plt.subplots()
ax.plot(cpu_td["timestamp"], cpu_td["cpu"], label="CPU usage", color="black", marker="o", linestyle="--")
ax.set_xlabel("time")
ax.set_ylabel("CPU usage (%)")
ax.grid()
ax.yaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax2 = ax.twinx()
ax2.set_ylabel("users")
ax2.spines["right"].set_position(("axes", 1.025))
ax2.plot(user_td["timestamp"], user_td["publishers"], label="users", color="red", marker="o", linestyle="-")
ax3 = ax.twinx()
ax3.set_ylabel("bitrate")
ax3.spines["right"].set_position(("axes", 1.075))
ax3.plot(plt_remote_audio_inbound["timestamp"], plt_remote_audio_inbound["roundTripTime"], label="roundTripTime", linestyle="-")

fig.legend()
fig.suptitle("Inbound audio RTT (s) over time")

ax3.axhline(y=0.1, color="green", linestyle="-", label="good rtt")
ax3.axhline(y=0.2, color="orange", linestyle="-", label="mediocre rtt")

In [ ]:
# list of directories in stats/
dirs = os.listdir("stats/")
# read all events.json in each subdir
events = []
for i, dir in enumerate(dirs):
    subdirs = os.listdir(f"stats/{dir}")
    for subdir in subdirs:
        with open(f"stats/{dir}/{subdir}/events.json") as f:
            events.extend(json.load(f))

In [ ]:
connection_start_fn = lambda x: (x['event'] == 'connectionStart')
connection = list(filter(connection_start_fn, events))

connection_df = pd.DataFrame(columns=["user", "session", "time to connect (s)", "connection start time", "stream created time"])
for i, conn in enumerate(connection):
    participant = conn['participant']
    session = conn['session']
    connection_start = conn['timestamp']
    stream_created = list(filter(lambda x: (x['event'] == 'streamCreated') and (x['connection'] == 'local') and (x['participant'] == participant) and (x['session'] == session), events))
    if len(stream_created) > 1:
        print(f"More than one stream created for connection {participant}-{session}")
    if len(stream_created) == 0:
        print(f"No stream created for connection {participant}-{session}")
        continue
    created_time = stream_created[0]['timestamp']
    # convert to datetime
    connection_start = datetime.strptime(connection_start, '%Y-%m-%dT%H:%M:%S.%fZ')
    created_time = datetime.strptime(created_time, '%Y-%m-%dT%H:%M:%S.%fZ')
    time_diff = (created_time - connection_start).total_seconds()
    connection_df.loc[i] = [participant, session, time_diff, connection_start, created_time]

#display(connection_df)

#bar chart of time to connect
plt.rcParams["figure.figsize"] = [25, 10]
fig, ax = plt.subplots()
ax.bar(connection_df["user"] + "-"+ connection_df["session"], connection_df["time to connect (s)"])
ax.set_xlabel("user-session")
ax.set_ylabel("time to connect (s)")
# rotate x labels
plt.xticks(rotation=90)

ax.grid()
fig.suptitle("Time to publish for each connection (s)")